# Transformación inicial de los resultados electorales

Este es primer cuaderno dedicado al ETL de los datos que utilizamos en la Práctica Final. Consiste en un primer tratamiento de los datos extraidos directamente de la web del Ministerio del Interior.

Utilizaremos, para cada elección, los ficheros del Ministerio que especifican los datos comunes a las mesas electorales, los datos de cada uno de los partidos, y, desde luego, los votos a los distintos partidos en cada mesa.

El resultado del cuaderno serán los resultados electorales en cada sección, que está formada siempre por una o varias mesas electorales.

Comenzamos numpy y pandas.

In [1]:
import numpy as np
import pandas as pd

In [144]:
#la primera vez...
!pip install boto3 

You should consider upgrading via the 'c:\users\evaes\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import boto3

BUCKET_NAME = 'electomedia' 

# sustituir por credenciales de acceso. 
s3 = boto3.resource('s3', aws_access_key_id = 'xxxxxxxxxxxxx', 
                          aws_secret_access_key= 'xxxxxxxxxxxxxx')

In [3]:
import botocore.exceptions

KEY = 'INE/10021111.DAT' #carpeta en la que se encuentran los datos del INE

try:
  s3.Bucket(BUCKET_NAME).download_file(KEY, '10021111.DAT')
  
except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise
    


Cargamos inicialmente el fichero de los resultados electorales, en este caso los de las elecciones de noviembre 2011.

Vemos que tenemos más de 700 mil registros. Cada uno de ellos es el voto a un determinado partido en cada mesa; pueden ser cero, pues hay un registro con solo que un partido se haya presentado.

In [4]:
res_mesa= pd.read_csv('10021111.DAT', delimiter=' ', names=['Sección', 'Votación'])

In [5]:
res_mesa

,Sección,Votación
0,022011111010400101001,B0000870000018
1,022011111010400101001,B0001370000212
2,022011111010400201001,A0000050000000
3,022011111010400201001,A0000300000001
4,022011111010400301002,A0000050000001
...,...,...
758435,0220111111952999090000U0001270000037,NaN
758436,0220111111999999090000U0001280000037,NaN
758437,0220111119999999090000U0001130000000,NaN
758438,0220111119999999090000U0001240000014,NaN


Vemos que hay datos NaN, que se causan por que pandas no ha distinguido correctamente las columnas. Lo corregimos con el método slice.

In [6]:
pd.isna(res_mesa['Votación']).sum()

919

In [7]:
res_mesa['Votación'].fillna(res_mesa['Sección'].str.slice(start = 22), inplace = True)

In [8]:
pd.isna(res_mesa['Votación']).sum()

0

In [9]:
res_mesa['Sección'] = res_mesa['Sección'].str.slice(start = 0, stop = 21)

In [10]:
res_mesa

,Sección,Votación
0,022011111010400101001,B0000870000018
1,022011111010400101001,B0001370000212
2,022011111010400201001,A0000050000000
3,022011111010400201001,A0000300000001
4,022011111010400301002,A0000050000001
...,...,...
758435,022011111195299909000,U0001270000037
758436,022011111199999909000,U0001280000037
758437,022011111999999909000,U0001130000000
758438,022011111999999909000,U0001240000014


El Ministerio suministra unas instrucciones que permiten separar los datos de los partidos y los votos obtenidos. Las aplicamos, y definimos las columnas correspondientes.

In [11]:
res_mesa['Partido'] = res_mesa['Votación'].str[1:7].replace(np.nan, 0).astype(int)

In [12]:
res_mesa

,Sección,Votación,Partido
0,022011111010400101001,B0000870000018,87
1,022011111010400101001,B0001370000212,137
2,022011111010400201001,A0000050000000,5
3,022011111010400201001,A0000300000001,30
4,022011111010400301002,A0000050000001,5
...,...,...,...
758435,022011111195299909000,U0001270000037,127
758436,022011111199999909000,U0001280000037,128
758437,022011111999999909000,U0001130000000,113
758438,022011111999999909000,U0001240000014,124


In [13]:
res_mesa['Votos'] = res_mesa['Votación'].str[7:].replace(np.nan, 0).astype(int)

In [14]:
res_mesa

,Sección,Votación,Partido,Votos
0,022011111010400101001,B0000870000018,87,18
1,022011111010400101001,B0001370000212,137,212
2,022011111010400201001,A0000050000000,5,0
3,022011111010400201001,A0000300000001,30,1
4,022011111010400301002,A0000050000001,5,1
...,...,...,...,...
758435,022011111195299909000,U0001270000037,127,37
758436,022011111199999909000,U0001280000037,128,37
758437,022011111999999909000,U0001130000000,113,0
758438,022011111999999909000,U0001240000014,124,14


In [15]:
import botocore.exceptions

KEY = 'INE/03021111.DAT' 
try:
  
  s3.Bucket(BUCKET_NAME).download_file(KEY, '03021111.DAT')
  
except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

El Ministerio almacena los nombres de los partidos que concurren a las elecciones en un fichero, que lleemos en el dataframe partidos_elec.

In [16]:
partidos_elec = pd.read_csv('03021111.DAT', names=['Partido'], encoding = 'latin_1')

In [17]:
res_mesa['Votos'].sum()

24160535

In [18]:
partidos_elec

,Partido
0,02201111000001ACIMA ...
1,02201111000002ANC ...
2,02201111000003AMAIUR ...
3,02201111000004ANDECHA ...
4,02201111000005ANTICAPITALISTAS ...
...,...
122,02201111000162UCE ...
123,02201111000165U.C.I.T. ...
124,02201111000167UPN-PP ...
125,02201111000168UPyD ...


Obtenemos el código numérico de cada partido, pese a que no es muy útil, pues cambia cada elección, así como sus nombres y siglas oficiales. Lo complicado es que un mismo partido puede tener de facto nombres distintos según la provincia.

In [19]:
partidos_elec['Cod_Partido'] = partidos_elec['Partido'].str[8:14].replace(np.nan, 0, regex=True).astype(int)

In [20]:
partidos_elec

,Partido,Cod_Partido
0,02201111000001ACIMA ...,1
1,02201111000002ANC ...,2
2,02201111000003AMAIUR ...,3
3,02201111000004ANDECHA ...,4
4,02201111000005ANTICAPITALISTAS ...,5
...,...,...
122,02201111000162UCE ...,162
123,02201111000165U.C.I.T. ...,165
124,02201111000167UPN-PP ...,167
125,02201111000168UPyD ...,168


In [21]:
partidos_elec['Nom_Partido'] = partidos_elec['Partido'].str[64:214].replace(np.nan, 0, regex=True)
partidos_elec['Siglas_Partido'] = partidos_elec['Partido'].str[14:64].replace(np.nan, 0, regex=True)

In [22]:
partidos_elec

,Partido,Cod_Partido,Nom_Partido,Siglas_Partido
0,02201111000001ACIMA ...,1,ACCION CIUDADANA POR MALAGA ...,ACIMA ...
1,02201111000002ANC ...,2,ALTERNATIVA NACIONALISTA CANARIA ...,ANC ...
2,02201111000003AMAIUR ...,3,AMAIUR ...,AMAIUR ...
3,02201111000004ANDECHA ...,4,ANDECHA ASTUR ...,ANDECHA ...
4,02201111000005ANTICAPITALISTAS ...,5,ANTICAPITALISTAS ...,ANTICAPITALISTAS ...
...,...,...,...,...
122,02201111000162UCE ...,162,UNIFICACI-N COMUNISTA DE ESPA-A ...,UCE ...
123,02201111000165U.C.I.T. ...,165,UNION DE CIUDADANOS INDEPENDIENTES DE TOLEDO ...,U.C.I.T. ...
124,02201111000167UPN-PP ...,167,UNI-N DEL PUEBLO NAVARRO EN COALICI-N CON EL P...,UPN-PP ...
125,02201111000168UPyD ...,168,UNI-N PROGRESO Y DEMOCRACIA ...,UPyD ...


In [23]:
import botocore.exceptions

KEY = 'INE/Partidos.txt' 

try:
 
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'Partidos.txt')
  
except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

Por simplicidad, hemos definido los nombres, códigos y siglas de los principales partidos a lo largo de las elecciones entre 2011 y 2019, lo cual hemos guardado en un fichero csv, y que cargamos a continuación en el df partidos. Incluimos el código 25 ('otros') para los minoritarios.

In [24]:
partidos = pd.read_csv('Partidos.txt', names=['C_Partido', 'S_Partido', 'N_Partido'])

In [25]:
partidos

,C_Partido,S_Partido,N_Partido
0,1,PP,Partido Popular
1,2,PSOE,Partido Socialista Obrero Español
2,3,Cs,Ciudadanos
3,4,UP,Unidas Podemos
4,5,IU,Izquierda Unida
5,6,VOX,Vox
6,7,UPyD,Unión Progreso y Democracia
7,8,MP,Más País
8,9,CiU,Convergencia i Unió
9,10,ERC,Esquerra Republicana de Catalunya


Se trata ahora de asignar nuestros códigos a los partidos que se presentan en cada elección. Y ello hay que hacerlo uno a uno. Por comodidad, asignamos inicialmente el códido 25 a todos ellos, y nos quedamos con las columnas más importantes.

In [26]:
partidos_elec_simp = partidos_elec[['Cod_Partido', 'Siglas_Partido']]

In [27]:
partidos_elec_simp['Cod'] = 25

<ipython-input-27-ae14e9438eed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partidos_elec_simp['Cod'] = 25


In [28]:
partidos_elec_simp

,Cod_Partido,Siglas_Partido,Cod
0,1,ACIMA ...,25
1,2,ANC ...,25
2,3,AMAIUR ...,25
3,4,ANDECHA ...,25
4,5,ANTICAPITALISTAS ...,25
...,...,...,...
122,162,UCE ...,25
123,165,U.C.I.T. ...,25
124,167,UPN-PP ...,25
125,168,UPyD ...,25


Guardamos el df de los partidos que se presentaron en 2011, según Interior, y que tienen asignado de momento el código 25.

Cargando el fichero en VSCode vamos asignando los códigos correctos uno a uno, una labor nada divertida. Una vez que terminamos, cargamos de nuevo el fichero, ya con los códigos de partidos correctos.

In [29]:
partidos_elec_simp.to_csv('partidos_elec_simp_N11.txt', sep = ',', index = False)

In [31]:
import botocore.exceptions

KEY = 'INE/partidos_elec_simp_N11.txt' 

try:
  
  s3.Bucket(BUCKET_NAME).download_file(KEY, 'partidos_elec_simp_N11.txt')
  
except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

In [32]:
partidos_elec_simp = pd.read_csv('partidos_elec_simp_N11.txt')

In [33]:
partidos_elec_simp

,Cod_Partido,Siglas_Partido,Cod
0,1,ACIMA ...,25
1,2,ANC ...,25
2,3,AMAIUR ...,16
3,4,ANDECHA ...,25
4,5,ANTICAPITALISTAS ...,25
...,...,...,...
122,162,UCE ...,25
123,165,U.C.I.T. ...,25
124,167,UPN-PP ...,1
125,168,UPyD ...,7


Ahora hacemos el primer merge, entre los resultados de las mesas y el fichero de los partidos con su código correcto.

In [34]:
Resultados_elec = res_mesa.merge(partidos_elec_simp, left_on = 'Partido', right_on = 'Cod_Partido')

In [35]:
Resultados_elec

,Sección,Votación,Partido,Votos,Cod_Partido,Siglas_Partido,Cod
0,022011111010400101001,B0000870000018,87,18,87,IULV-CA ...,5
1,022011111010400303001,B0000870000009,87,9,87,IULV-CA ...,5
2,022011111010400304001,B0000870000004,87,4,87,IULV-CA ...,5
3,022011111010400602002,A0000870000015,87,15,87,IULV-CA ...,5
4,022011111010400301002,B0000870000009,87,9,87,IULV-CA ...,5
...,...,...,...,...,...,...,...
758435,022011111189999909000,U0000990000004,99,4,99,LV-GV ...,25
758436,022011111185100101001,B0000990000008,99,8,99,LV-GV ...,25
758437,022011111185100103008,U0000990000004,99,4,99,LV-GV ...,25
758438,022011111185100104011,A0000990000003,99,3,99,LV-GV ...,25


Hacemos comprobaciones de los datos, en este caso sumando los votos al PP, código 1, así como los votos totales. Salen bien.

In [36]:
Resultados_elec['Votos'].sum()

24160535

In [37]:
Resultados_elec.loc[Resultados_elec['Cod'] == 1]['Votos'].sum()

10921796

Hacemos otro merge, en esta caso para añadir el nombre del partido que hemos definido nosotros, y no el larguísimo que viene en fichero de los partidos del Ministerio.

In [38]:
Resultados_elec_aj = Resultados_elec.merge(partidos, left_on = 'Cod', right_on = 'C_Partido')

In [39]:
Resultados_elec_aj

,Sección,Votación,Partido,Votos,Cod_Partido,Siglas_Partido,Cod,C_Partido,S_Partido,N_Partido
0,022011111010400101001,B0000870000018,87,18,87,IULV-CA ...,5,5,IU,Izquierda Unida
1,022011111010400303001,B0000870000009,87,9,87,IULV-CA ...,5,5,IU,Izquierda Unida
2,022011111010400304001,B0000870000004,87,4,87,IULV-CA ...,5,5,IU,Izquierda Unida
3,022011111010400602002,A0000870000015,87,15,87,IULV-CA ...,5,5,IU,Izquierda Unida
4,022011111010400301002,B0000870000009,87,9,87,IULV-CA ...,5,5,IU,Izquierda Unida
...,...,...,...,...,...,...,...,...,...,...
758435,022011111144890203002,U0000540000416,54,416,54,EAJ-PNV ...,14,14,PNV,Partido Nacionalista Vasco
758436,022011111144890203003,U0000540000317,54,317,54,EAJ-PNV ...,14,14,PNV,Partido Nacionalista Vasco
758437,022011111144890501002,U0000540000361,54,361,54,EAJ-PNV ...,14,14,PNV,Partido Nacionalista Vasco
758438,022011111144891001003,A0000540000104,54,104,54,EAJ-PNV ...,14,14,PNV,Partido Nacionalista Vasco


In [40]:
Resultados_votos = Resultados_elec_aj[['Sección', 'Votación', 'Cod', 'S_Partido', 'Votos', 'Siglas_Partido', 'Partido']]

In [41]:
Resultados_votos

,Sección,Votación,Cod,S_Partido,Votos,Siglas_Partido,Partido
0,022011111010400101001,B0000870000018,5,IU,18,IULV-CA ...,87
1,022011111010400303001,B0000870000009,5,IU,9,IULV-CA ...,87
2,022011111010400304001,B0000870000004,5,IU,4,IULV-CA ...,87
3,022011111010400602002,A0000870000015,5,IU,15,IULV-CA ...,87
4,022011111010400301002,B0000870000009,5,IU,9,IULV-CA ...,87
...,...,...,...,...,...,...,...
758435,022011111144890203002,U0000540000416,14,PNV,416,EAJ-PNV ...,54
758436,022011111144890203003,U0000540000317,14,PNV,317,EAJ-PNV ...,54
758437,022011111144890501002,U0000540000361,14,PNV,361,EAJ-PNV ...,54
758438,022011111144891001003,A0000540000104,14,PNV,104,EAJ-PNV ...,54


In [42]:
Resultados_votos['Votos'].sum()

24160535

In [43]:
pd.isna(Resultados_votos['Votos']).sum()

0

Llega la primera transformación importante, y es que los votos de los españoles que viven en el extranjero (voto CERA) no nos interesan, porque por definición no le podemos relacionar con variables de tipo socioeconómico en España.

Para eliminar estos datos, extraemos el código del municipio de cada sección. Los votos CERA están designados por el Ministerio como del municipio 999, por lo que nos quedamos sencillamente con el resto.

In [44]:
Resultados_votos['Municipio'] = Resultados_votos['Sección'].str[13:16]

<ipython-input-44-91a4cbca1d4b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Resultados_votos['Municipio'] = Resultados_votos['Sección'].str[13:16]


In [45]:
Resultados_votos

,Sección,Votación,Cod,S_Partido,Votos,Siglas_Partido,Partido,Municipio
0,022011111010400101001,B0000870000018,5,IU,18,IULV-CA ...,87,001
1,022011111010400303001,B0000870000009,5,IU,9,IULV-CA ...,87,003
2,022011111010400304001,B0000870000004,5,IU,4,IULV-CA ...,87,003
3,022011111010400602002,A0000870000015,5,IU,15,IULV-CA ...,87,006
4,022011111010400301002,B0000870000009,5,IU,9,IULV-CA ...,87,003
...,...,...,...,...,...,...,...,...
758435,022011111144890203002,U0000540000416,14,PNV,416,EAJ-PNV ...,54,902
758436,022011111144890203003,U0000540000317,14,PNV,317,EAJ-PNV ...,54,902
758437,022011111144890501002,U0000540000361,14,PNV,361,EAJ-PNV ...,54,905
758438,022011111144891001003,A0000540000104,14,PNV,104,EAJ-PNV ...,54,910


In [46]:
Resultados_votos = Resultados_votos.loc[Resultados_votos['Municipio'] != '999', :]

In [47]:
Resultados_votos

,Sección,Votación,Cod,S_Partido,Votos,Siglas_Partido,Partido,Municipio
0,022011111010400101001,B0000870000018,5,IU,18,IULV-CA ...,87,001
1,022011111010400303001,B0000870000009,5,IU,9,IULV-CA ...,87,003
2,022011111010400304001,B0000870000004,5,IU,4,IULV-CA ...,87,003
3,022011111010400602002,A0000870000015,5,IU,15,IULV-CA ...,87,006
4,022011111010400301002,B0000870000009,5,IU,9,IULV-CA ...,87,003
...,...,...,...,...,...,...,...,...
758435,022011111144890203002,U0000540000416,14,PNV,416,EAJ-PNV ...,54,902
758436,022011111144890203003,U0000540000317,14,PNV,317,EAJ-PNV ...,54,902
758437,022011111144890501002,U0000540000361,14,PNV,361,EAJ-PNV ...,54,905
758438,022011111144891001003,A0000540000104,14,PNV,104,EAJ-PNV ...,54,910


In [48]:
Resultados_votos['Sección'].unique().shape

(35957,)

In [49]:
Resultados_votos['Votos'].sum()

23942870

LLegamos a otro paso importante: tenemos que poner los votos a cada partido como columnas, luego hacemos un pivot. Los NaN aparecen porque cada registro es el voto a UN determinado partido, por lo que, por definición, solo una columna no será NaN. Sustituímos a este signo con ceros.

In [50]:
a = Resultados_votos.pivot(columns='S_Partido', values='Votos')

In [51]:
a

S_Partido,Amaiur,BNG,CC,CiU,ERC,GBai,IU,Otros,PACMA,PNV,PP,PRC,PSOE,UPyD
0,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,416.0,NaN,NaN,NaN,NaN
758436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317.0,NaN,NaN,NaN,NaN
758437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,361.0,NaN,NaN,NaN,NaN
758438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,NaN,NaN,NaN,NaN


In [52]:
a.fillna(0)

S_Partido,Amaiur,BNG,CC,CiU,ERC,GBai,IU,Otros,PACMA,PNV,PP,PRC,PSOE,UPyD
0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,416.0,0.0,0.0,0.0,0.0
758436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,317.0,0.0,0.0,0.0,0.0
758437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,361.0,0.0,0.0,0.0,0.0
758438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,0.0,0.0,0.0,0.0


Ahora viene otro paso clave: agrupamos los registros por secciones y sumamo. Así obtenemos los resultados de cada sección. Tendremos ahora unas 36 mil secciones como registros.

Pasamos los datos a tipo entero, pues es lo que son en la vida real, y hacemos un reset del index.

In [53]:
a['Sección'] = Resultados_votos['Sección']

In [54]:
b = a.groupby(by=["Sección"]).sum()

In [55]:
b

S_Partido,Amaiur,BNG,CC,CiU,ERC,GBai,IU,Otros,PACMA,PNV,PP,PRC,PSOE,UPyD
Sección,,,,,,,,,,,,,,
022011111010400101001,0.0,0.0,0.0,0.0,0.0,0.0,34.0,10.0,0.0,0.0,434.0,0.0,373.0,27.0
022011111010400201001,0.0,0.0,0.0,0.0,0.0,0.0,40.0,9.0,0.0,0.0,361.0,0.0,454.0,19.0
022011111010400301001,0.0,0.0,0.0,0.0,0.0,0.0,13.0,6.0,0.0,0.0,364.0,0.0,129.0,14.0
022011111010400301002,0.0,0.0,0.0,0.0,0.0,0.0,16.0,18.0,0.0,0.0,547.0,0.0,223.0,23.0
022011111010400301003,0.0,0.0,0.0,0.0,0.0,0.0,35.0,7.0,0.0,0.0,598.0,0.0,195.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
022011111195200108010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,3.0,0.0,541.0,0.0,146.0,43.0
022011111195200108011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,3.0,0.0,535.0,0.0,187.0,44.0
022011111195200108012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,5.0,0.0,843.0,0.0,245.0,37.0


In [56]:
b['PSOE'].sum()

6975407.0

In [57]:
b.reset_index()

S_Partido,Sección,Amaiur,BNG,CC,CiU,ERC,GBai,IU,Otros,PACMA,PNV,PP,PRC,PSOE,UPyD
0,022011111010400101001,0.0,0.0,0.0,0.0,0.0,0.0,34.0,10.0,0.0,0.0,434.0,0.0,373.0,27.0
1,022011111010400201001,0.0,0.0,0.0,0.0,0.0,0.0,40.0,9.0,0.0,0.0,361.0,0.0,454.0,19.0
2,022011111010400301001,0.0,0.0,0.0,0.0,0.0,0.0,13.0,6.0,0.0,0.0,364.0,0.0,129.0,14.0
3,022011111010400301002,0.0,0.0,0.0,0.0,0.0,0.0,16.0,18.0,0.0,0.0,547.0,0.0,223.0,23.0
4,022011111010400301003,0.0,0.0,0.0,0.0,0.0,0.0,35.0,7.0,0.0,0.0,598.0,0.0,195.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35952,022011111195200108010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,3.0,0.0,541.0,0.0,146.0,43.0
35953,022011111195200108011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,3.0,0.0,535.0,0.0,187.0,44.0
35954,022011111195200108012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,5.0,0.0,843.0,0.0,245.0,37.0
35955,022011111195200108013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,0.0,415.0,0.0,167.0,21.0


In [58]:
c = b.astype('int32')

In [59]:
c

S_Partido,Amaiur,BNG,CC,CiU,ERC,GBai,IU,Otros,PACMA,PNV,PP,PRC,PSOE,UPyD
Sección,,,,,,,,,,,,,,
022011111010400101001,0,0,0,0,0,0,34,10,0,0,434,0,373,27
022011111010400201001,0,0,0,0,0,0,40,9,0,0,361,0,454,19
022011111010400301001,0,0,0,0,0,0,13,6,0,0,364,0,129,14
022011111010400301002,0,0,0,0,0,0,16,18,0,0,547,0,223,23
022011111010400301003,0,0,0,0,0,0,35,7,0,0,598,0,195,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
022011111195200108010,0,0,0,0,0,0,0,19,3,0,541,0,146,43
022011111195200108011,0,0,0,0,0,0,0,19,3,0,535,0,187,44
022011111195200108012,0,0,0,0,0,0,0,30,5,0,843,0,245,37


In [60]:
d = c.reset_index()

In [61]:
d

S_Partido,Sección,Amaiur,BNG,CC,CiU,ERC,GBai,IU,Otros,PACMA,PNV,PP,PRC,PSOE,UPyD
0,022011111010400101001,0,0,0,0,0,0,34,10,0,0,434,0,373,27
1,022011111010400201001,0,0,0,0,0,0,40,9,0,0,361,0,454,19
2,022011111010400301001,0,0,0,0,0,0,13,6,0,0,364,0,129,14
3,022011111010400301002,0,0,0,0,0,0,16,18,0,0,547,0,223,23
4,022011111010400301003,0,0,0,0,0,0,35,7,0,0,598,0,195,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35952,022011111195200108010,0,0,0,0,0,0,0,19,3,0,541,0,146,43
35953,022011111195200108011,0,0,0,0,0,0,0,19,3,0,535,0,187,44
35954,022011111195200108012,0,0,0,0,0,0,0,30,5,0,843,0,245,37
35955,022011111195200108013,0,0,0,0,0,0,0,12,2,0,415,0,167,21


In [62]:
import botocore.exceptions

KEY = 'INE/09021111.DAT' 

try:
 
  s3.Bucket(BUCKET_NAME).download_file(KEY, '09021111.DAT')
  
except botocore.exceptions.ClientError as e:
  if e.response['Error']['Code'] == "404":
    print("The object does not exist.")
  else:
    raise

Nos faltan aún los datos comunes a cada sección en cada elección, es decir, su censo electoral, cuánta gente votó, los votos nulos y los blancos, etc.. Para ello cargamos el fichero del Ministerio que almacena esos datos para cada mesa electoral. En 2011 hubo casi 60 mil mesas electorales en cerca de 36 mil secciones.

In [63]:
datos_votacion_elec = pd.read_csv('09021111.DAT', delimiter=' ', names=['Sección', 'Votación'])

In [64]:
datos_votacion_elec

,Sección,Votación
0,022011111010400301003,A000057300005730000000000000000002270000330000...
1,022011111010400301003,B000064500006450000000000000000002560000389000...
2,022011111010400302002,B000055100005510000000000000000002130000305000...
3,022011111010400304001,B000019100001910000000000000000000810000126000...
4,022011111010400601002,B000072100007210000000000000000002890000418000...
...,...,...
59943,022011111195200107003,C000047200004720000000000000000001620000233000...
59944,022011111195200107004,A000059600005960000000000000000001740000265000...
59945,022011111195200108002,U000082400008240000000000000000002050000314000...
59946,022011111195200108005,B000064200006420000000000000000002340000333000...


De nuevo tenemos que corregir los registros en blanco, que se trata normalmente de los datos de los votos CERA, que por cierto en unas elecciones generales no se asignan a ninguna seccion en concreto sino a las provincias.

In [65]:
pd.isna(datos_votacion_elec['Votación']).sum()

72

In [66]:
datos_votacion_elec['Votación'].fillna(datos_votacion_elec['Sección'].str.slice(start = 22), inplace = True)

In [67]:
datos_votacion_elec.loc[pd.isna(datos_votacion_elec['Votación'])]

,Sección,Votación


In [68]:
datos_votacion_elec['Sección'] = datos_votacion_elec['Sección'].str.slice(start = 0, stop = 21)

In [69]:
datos_votacion_elec

,Sección,Votación
0,022011111010400301003,A000057300005730000000000000000002270000330000...
1,022011111010400301003,B000064500006450000000000000000002560000389000...
2,022011111010400302002,B000055100005510000000000000000002130000305000...
3,022011111010400304001,B000019100001910000000000000000000810000126000...
4,022011111010400601002,B000072100007210000000000000000002890000418000...
...,...,...
59943,022011111195200107003,C000047200004720000000000000000001620000233000...
59944,022011111195200107004,A000059600005960000000000000000001740000265000...
59945,022011111195200108002,U000082400008240000000000000000002050000314000...
59946,022011111195200108005,B000064200006420000000000000000002340000333000...


Siguiendo las instrucciones del Ministerio, vamos sacando datos de las votaciones en cada mesa electoral. Al final solo eligiremos los que realmente necesitemos.

In [70]:
datos_votacion_elec['Elección'] = datos_votacion_elec['Sección'].str[0:2]
datos_votacion_elec['Año'] = datos_votacion_elec['Sección'].str[2:6]

In [71]:
datos_votacion_elec

,Sección,Votación,Elección,Año
0,022011111010400301003,A000057300005730000000000000000002270000330000...,02,2011
1,022011111010400301003,B000064500006450000000000000000002560000389000...,02,2011
2,022011111010400302002,B000055100005510000000000000000002130000305000...,02,2011
3,022011111010400304001,B000019100001910000000000000000000810000126000...,02,2011
4,022011111010400601002,B000072100007210000000000000000002890000418000...,02,2011
...,...,...,...,...
59943,022011111195200107003,C000047200004720000000000000000001620000233000...,02,2011
59944,022011111195200107004,A000059600005960000000000000000001740000265000...,02,2011
59945,022011111195200108002,U000082400008240000000000000000002050000314000...,02,2011
59946,022011111195200108005,B000064200006420000000000000000002340000333000...,02,2011


In [72]:
datos_votacion_elec['Mes'] = datos_votacion_elec['Sección'].str[6:8]
datos_votacion_elec['CCAA'] = datos_votacion_elec['Sección'].str[9:11]
datos_votacion_elec['Provincia'] = datos_votacion_elec['Sección'].str[11:13]
datos_votacion_elec['Municipio'] = datos_votacion_elec['Sección'].str[13:16]
datos_votacion_elec['Sección_ID'] = datos_votacion_elec['Sección'].str[18:21]

In [73]:
datos_votacion_elec

,Sección,Votación,Elección,Año,Mes,CCAA,Provincia,Municipio,Sección_ID
0,022011111010400301003,A000057300005730000000000000000002270000330000...,02,2011,11,01,04,003,003
1,022011111010400301003,B000064500006450000000000000000002560000389000...,02,2011,11,01,04,003,003
2,022011111010400302002,B000055100005510000000000000000002130000305000...,02,2011,11,01,04,003,002
3,022011111010400304001,B000019100001910000000000000000000810000126000...,02,2011,11,01,04,003,001
4,022011111010400601002,B000072100007210000000000000000002890000418000...,02,2011,11,01,04,006,002
...,...,...,...,...,...,...,...,...,...
59943,022011111195200107003,C000047200004720000000000000000001620000233000...,02,2011,11,19,52,001,003
59944,022011111195200107004,A000059600005960000000000000000001740000265000...,02,2011,11,19,52,001,004
59945,022011111195200108002,U000082400008240000000000000000002050000314000...,02,2011,11,19,52,001,002
59946,022011111195200108005,B000064200006420000000000000000002340000333000...,02,2011,11,19,52,001,005


In [74]:
datos_votacion_elec['Censo'] = datos_votacion_elec['Votación'].str[1:8].replace(np.nan, 0, regex=True).astype(int)
datos_votacion_elec['Censo_Esc'] = datos_votacion_elec['Votación'].str[8:15].replace(np.nan, 0, regex=True).astype(int)
datos_votacion_elec['Censo_CERE'] = datos_votacion_elec['Votación'].str[15:22].replace(np.nan, 0, regex=True).astype(int)
datos_votacion_elec['Vot_CERE'] = datos_votacion_elec['Votación'].str[22:29].replace(np.nan, 0, regex=True).astype(int)
datos_votacion_elec['Blanco'] = datos_votacion_elec['Votación'].str[43:50].replace(np.nan, 0, regex=True).astype(int)
datos_votacion_elec['Nulos'] = datos_votacion_elec['Votación'].str[50:57].replace(np.nan, 0, regex=True).astype(int)
datos_votacion_elec['V_Cand'] = datos_votacion_elec['Votación'].str[57:64].replace(np.nan, 0, regex=True).astype(int)

In [75]:
datos_votacion_elec

,Sección,Votación,Elección,Año,Mes,CCAA,Provincia,Municipio,Sección_ID,Censo,Censo_Esc,Censo_CERE,Vot_CERE,Blanco,Nulos,V_Cand
0,022011111010400301003,A000057300005730000000000000000002270000330000...,02,2011,11,01,04,003,003,573,573,0,0,3,8,404
1,022011111010400301003,B000064500006450000000000000000002560000389000...,02,2011,11,01,04,003,003,645,645,0,0,6,4,459
2,022011111010400302002,B000055100005510000000000000000002130000305000...,02,2011,11,01,04,003,002,551,551,0,0,2,3,360
3,022011111010400304001,B000019100001910000000000000000000810000126000...,02,2011,11,01,04,003,001,191,191,0,0,0,0,136
4,022011111010400601002,B000072100007210000000000000000002890000418000...,02,2011,11,01,04,006,002,721,721,0,0,8,9,497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59943,022011111195200107003,C000047200004720000000000000000001620000233000...,02,2011,11,19,52,001,003,472,472,0,0,2,4,297
59944,022011111195200107004,A000059600005960000000000000000001740000265000...,02,2011,11,19,52,001,004,596,596,0,0,5,5,352
59945,022011111195200108002,U000082400008240000000000000000002050000314000...,02,2011,11,19,52,001,002,824,824,0,0,9,2,374
59946,022011111195200108005,B000064200006420000000000000000002340000333000...,02,2011,11,19,52,001,005,642,642,0,0,6,2,422


Como hicimos antes, eliminamos los registros del voto CERA.

In [76]:
datos_votacion_elec = datos_votacion_elec.loc[datos_votacion_elec['Municipio'] != '999', :]

In [77]:
datos_votacion_elec

,Sección,Votación,Elección,Año,Mes,CCAA,Provincia,Municipio,Sección_ID,Censo,Censo_Esc,Censo_CERE,Vot_CERE,Blanco,Nulos,V_Cand
0,022011111010400301003,A000057300005730000000000000000002270000330000...,02,2011,11,01,04,003,003,573,573,0,0,3,8,404
1,022011111010400301003,B000064500006450000000000000000002560000389000...,02,2011,11,01,04,003,003,645,645,0,0,6,4,459
2,022011111010400302002,B000055100005510000000000000000002130000305000...,02,2011,11,01,04,003,002,551,551,0,0,2,3,360
3,022011111010400304001,B000019100001910000000000000000000810000126000...,02,2011,11,01,04,003,001,191,191,0,0,0,0,136
4,022011111010400601002,B000072100007210000000000000000002890000418000...,02,2011,11,01,04,006,002,721,721,0,0,8,9,497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59943,022011111195200107003,C000047200004720000000000000000001620000233000...,02,2011,11,19,52,001,003,472,472,0,0,2,4,297
59944,022011111195200107004,A000059600005960000000000000000001740000265000...,02,2011,11,19,52,001,004,596,596,0,0,5,5,352
59945,022011111195200108002,U000082400008240000000000000000002050000314000...,02,2011,11,19,52,001,002,824,824,0,0,9,2,374
59946,022011111195200108005,B000064200006420000000000000000002340000333000...,02,2011,11,19,52,001,005,642,642,0,0,6,2,422


In [78]:
datos_votacion_elec['Censo_Esc'].sum()

34296907

Definimos una columna con los votos totales, que nos será útil para calcular la participación, por ejemplo.

In [79]:
datos_votacion_elec['Votos_Total'] = datos_votacion_elec['Blanco'] + datos_votacion_elec['Nulos'] + datos_votacion_elec['V_Cand']

<ipython-input-79-b7548401b086>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_votacion_elec['Votos_Total'] = datos_votacion_elec['Blanco'] + datos_votacion_elec['Nulos'] + datos_votacion_elec['V_Cand']


In [80]:
datos_votacion_elec

,Sección,Votación,Elección,Año,Mes,CCAA,Provincia,Municipio,Sección_ID,Censo,Censo_Esc,Censo_CERE,Vot_CERE,Blanco,Nulos,V_Cand,Votos_Total
0,022011111010400301003,A000057300005730000000000000000002270000330000...,02,2011,11,01,04,003,003,573,573,0,0,3,8,404,415
1,022011111010400301003,B000064500006450000000000000000002560000389000...,02,2011,11,01,04,003,003,645,645,0,0,6,4,459,469
2,022011111010400302002,B000055100005510000000000000000002130000305000...,02,2011,11,01,04,003,002,551,551,0,0,2,3,360,365
3,022011111010400304001,B000019100001910000000000000000000810000126000...,02,2011,11,01,04,003,001,191,191,0,0,0,0,136,136
4,022011111010400601002,B000072100007210000000000000000002890000418000...,02,2011,11,01,04,006,002,721,721,0,0,8,9,497,514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59943,022011111195200107003,C000047200004720000000000000000001620000233000...,02,2011,11,19,52,001,003,472,472,0,0,2,4,297,303
59944,022011111195200107004,A000059600005960000000000000000001740000265000...,02,2011,11,19,52,001,004,596,596,0,0,5,5,352,362
59945,022011111195200108002,U000082400008240000000000000000002050000314000...,02,2011,11,19,52,001,002,824,824,0,0,9,2,374,385
59946,022011111195200108005,B000064200006420000000000000000002340000333000...,02,2011,11,19,52,001,005,642,642,0,0,6,2,422,430


In [81]:
datos_votacion_elec['Votos_Total'].sum()

24593080

Grabamos el df de los datos de cada mesa electoral. De todas formas, este fichero no utilizaremos, solo unas columnas de él.

In [82]:
datos_votacion_elec.to_csv('datos_votacion_elec_N11.txt', sep = ',', index = False)

Seleccionamos las columnas que realmente nos interesan, el censo, los votos totales, los votos blancos y nulos y los votos a candidaturas. La sección la mantenemos, pues tendremos que hacer el merge con ella.

In [83]:
datos_votacion_elec_aux = datos_votacion_elec[['Sección', 'Censo_Esc', 'Votos_Total', 'Nulos', 'Blanco', 'V_Cand']]

In [84]:
datos_votacion_elec_aux

,Sección,Censo_Esc,Votos_Total,Nulos,Blanco,V_Cand
0,022011111010400301003,573,415,8,3,404
1,022011111010400301003,645,469,4,6,459
2,022011111010400302002,551,365,3,2,360
3,022011111010400304001,191,136,0,0,136
4,022011111010400601002,721,514,9,8,497
...,...,...,...,...,...,...
59943,022011111195200107003,472,303,4,2,297
59944,022011111195200107004,596,362,5,5,352
59945,022011111195200108002,824,385,2,9,374
59946,022011111195200108005,642,430,2,6,422


Ahora agrupamos los datos de las mesas electorales por secciones, y sumamos. Pasamos a tener registros de las casi 36 mil secciones.

In [85]:
e = datos_votacion_elec_aux.groupby('Sección').sum()

In [86]:
e

,Censo_Esc,Votos_Total,Nulos,Blanco,V_Cand
Sección,,,,,
022011111010400101001,1139,890,6,6,878
022011111010400201001,1146,891,7,1,883
022011111010400301001,696,541,7,8,526
022011111010400301002,1159,840,7,6,827
022011111010400301003,1218,884,12,9,863
...,...,...,...,...,...
022011111195200108010,1208,775,8,15,752
022011111195200108011,1374,811,8,15,788
022011111195200108012,1874,1183,13,10,1160


In [87]:
e.reset_index()

,Sección,Censo_Esc,Votos_Total,Nulos,Blanco,V_Cand
0,022011111010400101001,1139,890,6,6,878
1,022011111010400201001,1146,891,7,1,883
2,022011111010400301001,696,541,7,8,526
3,022011111010400301002,1159,840,7,6,827
4,022011111010400301003,1218,884,12,9,863
...,...,...,...,...,...,...
35952,022011111195200108010,1208,775,8,15,752
35953,022011111195200108011,1374,811,8,15,788
35954,022011111195200108012,1874,1183,13,10,1160
35955,022011111195200108013,1167,637,7,13,617


Finalmente hacemos el merge de los datos de las elecciones (lo que acabamos de ver), con los datos de las votaciones a los partidos (el anterior). Así ya tenemos los datos completos de las elecciones por secciones, en este caso para las generales de 2011.

In [88]:
resultados_comp_elec = e.merge(d, left_on='Sección', right_on='Sección', suffixes=('Sección', 'Sección_ad'))

In [89]:
resultados_comp_elec

,Sección,Censo_Esc,Votos_Total,Nulos,Blanco,V_Cand,Amaiur,BNG,CC,CiU,ERC,GBai,IU,Otros,PACMA,PNV,PP,PRC,PSOE,UPyD
0,022011111010400101001,1139,890,6,6,878,0,0,0,0,0,0,34,10,0,0,434,0,373,27
1,022011111010400201001,1146,891,7,1,883,0,0,0,0,0,0,40,9,0,0,361,0,454,19
2,022011111010400301001,696,541,7,8,526,0,0,0,0,0,0,13,6,0,0,364,0,129,14
3,022011111010400301002,1159,840,7,6,827,0,0,0,0,0,0,16,18,0,0,547,0,223,23
4,022011111010400301003,1218,884,12,9,863,0,0,0,0,0,0,35,7,0,0,598,0,195,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35952,022011111195200108010,1208,775,8,15,752,0,0,0,0,0,0,0,19,3,0,541,0,146,43
35953,022011111195200108011,1374,811,8,15,788,0,0,0,0,0,0,0,19,3,0,535,0,187,44
35954,022011111195200108012,1874,1183,13,10,1160,0,0,0,0,0,0,0,30,5,0,843,0,245,37
35955,022011111195200108013,1167,637,7,13,617,0,0,0,0,0,0,0,12,2,0,415,0,167,21


Coprobamos para el primer registro que los votos a los distintos partidos concuerda con los votos a las candidaturas, como debe ser.

In [90]:
resultados_comp_elec.iloc[0, 6:].sum()

878

In [91]:
resultados_comp_elec['Votos_Total'].sum()

24593080

Guardamos el dataframe de los resultados de cada sección para la eleccion de 2011, en el fichero csv "resultados_comp_N11.txt", y así hacemos de forma análoga con las de 2015, 2016, y las dos de 2019.

In [92]:
resultados_comp_elec.to_csv('resultados_comp_N11.txt', sep = ',', index = False)

Con todo, hay que tener en cuenta que, por definición, solo tenemos en este df los partidos que se presentaron en 2011. En los df de otras elecciones las columnas serán distintas. Para que las columnas sean las mismas en todas las elecciones tendremos que reformar (uniformizar) los dataframes, y eso es lo que haremos en el siguiente cuaderno.